# PHASE 2 PROJECT
**Presented by:** Group 4.4
***

## Dennis Mwanzia 
## Pamela Awino 
## Ian Macharia
## Samuel Igecha
## Pauline Njeri

# 1.0  Project Introduction and Overview

The aim of this project is to put into practice our newly aqcuired skills in data science to formulate and solve a real business problem. In this project, we are given a raw dataset which is the house sales prices in Northwestern County and we are supposed to formulate a real business case study and use the data to solve the business problem and provide a useful business advisory which can be used to formulate real and valuable business decisions. Unlike in phase 1, we are not given the business problem, its upon us to formulate it.

## 1.1 Data Understanding

In [1]:
# Import libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

%matplotlib inline 
import seaborn as sns
plt.style.use('seaborn')

import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy import stats

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from scipy.stats import pearsonr
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler



import pickle
import json

import folium 
from folium import plugins
from folium.plugins import HeatMap


from my_functions import clean_data, round_bathrooms, remove_outliers, x_coord, fit_simple_linear_reg, fit_multiple_linear_reg, calculate_rmse, fit_polynomial_reg, plot_partial_residuals_all


ImportError: cannot import name 'plot_partial_residuals_all' from 'my_functions' (C:\Users\USER PC\Documents\Flatiron\WataalamAnalytics\wataalam-analytics--kings-developers-project\my_functions.py)

We have been provided with King County House Sales dataset found in 'kc_house_data.csv'. There is a description of columns dataset found in the md folder 'column_names.md'. Therefore, we examine the column descriptions to understand their meaning.

In [ ]:
#Unpacking the description 
with open('data/column_names.md', 'r') as f:
    md_text = f.read()

print(md_text)

In this problem, we have been given a dataset that contains the following variables;
1. id - unique identified for a house
2. dateDate - house was sold
3. pricePrice - is prediction target
4. bedroomsNumber - of Bedrooms/House
5. bathroomsNumber - of bathrooms/bedrooms
6. sqft_livingsquare - footage of the home
7. sqft_lotsquare - footage of the lot
8. floorsTotal - floors (levels) in house
9. waterfront - House which has a view to a waterfront
10. view - Has been viewed
11. condition - How good the condition is ( Overall )
12. grade - overall grade given to the housing unit, based on King County grading system
13. sqft_above - square footage of house apart from basement
14. sqft_basement - square footage of the basement
15. yr_built - Built Year
16. yr_renovated - Year when house was renovated
17. zipcode - zip
18. lat - Latitude coordinate
19. long - Longitude coordinate
20. sqft_living15 - The square footage of interior housing living space for the nearest 15 neighbors
21. sqft_lot15 - The square footage of the land lots of the nearest 15 neighbors

## Business Understanding

## Business Problem

The business problem is to provide guidance to Kings Wajenzi Developers, a prospective real estate developer in King County, on the most profitable types of properties to target in the current market. The guidance will include recommendations on optimal locations to develop properties, features most desired by customers and have the greatest impact on prices of properties, and season of year when homeowners are most likely to purchase properties. Our objective as Wataalamu Analytics Advisors is to analyze house sales data in King County which was collected between May 2014 and May 2015 and leverage the insights generated to guide the developer in selecting the most profitable properties to develop and maximize their profits. With King County's population on the rise and income stabilizing after the COVID-19 pandemic, demand for new homes in the area is at an all-time high.

## 1.2 Objectives

* To determine whether the time of year affects the price of a house and identify the most profitable seasons/months.

* To investigate whether location affects house prices in King County and identify the areas that attract the highest prices.

* To determine which features/attributes have the highest impact on the sales price of houses/properties in King County and identify the features that developers should focus on when developing upcoming projects in the area.

### Guiding Questions
####### What to answer

1. Does the time of the year affect the price of an house? If yes, which seasons/Months are most profitable?
2. Does Location affect prices? If yes, which locations within King county attract highest price of houses?
3. Which features/Attributes have the highest impact on sales price of houses/properties? Which features should the developer focus on when developing their upcoming projects in King County?


## Methodology

##### This project follows the OCEMiN Data Science framework for data analysis
1.	Obtain/import data
* We will import data from King County House Data provided as csv, using Pandas library.
2.	Clean/ Scrub data
* We will employ several data cleaning methods to modify and prepare the dataset for analysis.
3.	Explore data/EDA
* Analyzing and visualizing the cleaned data to gain insights, identify patterns and relationships, and formulate hypotheses.
4.	Model/Develop the predictive Model
*  Develop and evaluate predictive models using the data.
5.	Interpret Data
* Draw conclusions and make decisions based on the results of the analysis.

### 1.0 Importing Data

In [ ]:
df1 = pd.read_csv('data/kc_house_data.csv')
df1

In [ ]:
df1.shape

The dataset has 21597 rows and 21 columns

In [ ]:
df1.info()

The whole dataset is stored as pandas dataframe. We have three types of datatypes for columns which are integers, floats and objects. Since we are interested in running a linear regression model, we would like all the columns, to be numeric datatypes. However, some columns like `date`, `waterfront`, `view`, `condition`, `grade`, `sqft_basement` are objects and most likely categorical variables and will need to be transformed to dummy variables using one-hot encoding function of python. Intergers will also need to be examined to determine if they are binary variables or numeric data types. 
Its very important to note that sqft_basement should be of value type float based on the data preview done above. Therefore, we will need to investigate that column to find out why it appears as object instead of float.
The `date` column will need to be transformed to month and year columns as we seek to engineer a new feature named `seasons` to answer our first question.
The columns `waterfront`, `view`, `condition`, `grade` need to be investigated and necessary transformations done in order to make them dummy variables for our model. 

### 2.0 Data Cleaning

We will begin by checking the data for missing values. 

In [ ]:
#Checking for how many missing values
df1.isna().sum()

From the above output, we can see that Waterfront, view and year renovated have missing values. Lets check the percentage of missing values in the dataset. We can then calculate the percentage of missing values to see their magnitude and determine if we will drop the columns or just replace the missing values with appropriate replace values. 

In [ ]:
# Get percentage of missing values
for col in ['waterfront', 'view', 'yr_renovated']:
    percent_missing = df1[col].isnull().sum() * 100 / len(df1)
    print(f"{col}:{round(percent_missing,2)}")


`Waterfront`, `view` and `yr_renovated` columns have 11.0%, 0.29%, and 17.79% respectively of missing data. `Waterfront`, and `view` are of data type objects and therefore, we should replace the missing values with the mode. The `yr_renovated` shows the year in which renovations occurred and therefore, we should interpret the missing values to mean that no renovation occurred for those particular properties. Therefore, we will replace the missing values with zeros. 

Let us check the most common values for `waterfront` and `view` columns which we will use to replace the missing values. 

In [ ]:
print ("Waterfront Mode:",df1['waterfront'].mode()[0])
print ("view mode:", df1['view'].mode()[0])

Let us investigate the `sqft_basement` column to find out why it appeard as object data type whereas the vaalues inside show it should be Float data type.

In [ ]:
# Investigate sqft_basement column
df1['sqft_basement'].value_counts()

In [ ]:
# Get summary statistics for sqft_basement
df1['sqft_basement'].describe()

There is a significant number of entries with placeholder `?` as value for `sqft_basement`. Moreover, more than half of entries have a value of 0 indicating the properties did not have a basement and therefore, replace the placeholder with `0`. 

We discovered the data cleaning was occupying too much of our time and therefore created a function *clean_data* that we applied below. 

In [ ]:
df1 = clean_data(df1).copy()
df1

In [ ]:
# check if all missing values have been resolved
df1.isna().sum()

We no longer have any missing values, therefore we proceed to check for duplicates.

In [ ]:
#checking for duplicates

print("Number of duplicated values:", df1.duplicated().sum())

duplicate_rows = df1[df1.duplicated()]
duplicate_rows

We have two duplicates but upon closer examination, the two rows are not duplicates but rather may have arised as a result of data updates, data merging or entry errors.
Next, we can proceed to investigate the integer and object data types and perform the necessary transformations for our multiple regression model.

In [ ]:
# Confirming the data is clean and columns are of proper data types.
df1.info()

In [ ]:
# Investigatin bathroom column
# Get unique values
df1['bathrooms'].unique()

Bathrooms appear as float and we would expect bathrooms to be integers, so upon researching from common real estate jargon in the United states, bathrooms can be described with floats or fractions to indicate not a full bathroom or with an additional feature such as a sink. 

#### Checking for Outliers

After rigourous data clean up, we the try and finetune the data for ploting, visualization and subsequent modelling.

#### Boxplots

In [ ]:
# Select the variables you want to plot
cols_to_plot = df1.select_dtypes(include=['int64', 'float64']).columns

# Create a subplot grid
fig, axes = plt.subplots(nrows=1, ncols=len(cols_to_plot), figsize=(32, 6))

# Create a boxplot for each variable in a separate subplot
for i, col in enumerate(cols_to_plot):
    axes[i].boxplot(df1[col])
    axes[i].set_title(col)

plt.show()


In [ ]:
# Drop the irrelant columns for box plot purposes

# Select the categorical variables to drop and then generate boxplots
cols_to_plot = df1.select_dtypes(include=['int64', 'float64']).columns.drop([])

# Create a subplot grid
fig, axes = plt.subplots(nrows=1, ncols=len(cols_to_plot), figsize=(30, 6))

# Create a boxplot for each variable in a separate subplot
for i, col in enumerate(cols_to_plot):
    axes[i].boxplot(df1[col])
    axes[i].set_title(col)

plt.show()


In [ ]:
# Histogram plots for each data
# Select the numerical variables you want to plot
num_cols_to_plot = df1.select_dtypes(include=['int64', 'float64']).columns.drop([])
print("Columns to plot:",num_cols_to_plot )
# Create a histogram for each variable
df1[num_cols_to_plot].hist(figsize=(25, 12))
plt.savefig('Columnstoplot')
plt.show()


We dropped `view`, `grade`,`condition`, `zipcode` because they are categorical variables and therefore cannot be reasonably visualized using either box plots or histograms.
`Floors`, `age`, `year_Sold`, `month_sold` do not appear to have outliers. 
Therefore, we consider potential outliers in `bedrooms`, `bathrooms`, `sqft_living`, `sqft_lot`, `floors` and `sqft_above` features.

In [ ]:
# Investigate bedrooms
df1['bedrooms'].value_counts()

The houses with 33, 11 and 10 bedrooms need to be examined.

In [ ]:
bedroom_counts = [33, 11, 10]
columns_to_show = ['floors', 'price', 'sqft_living', 'waterfront', 'bathrooms', 'bedrooms']
houses_to_examine = df1[df1['bedrooms'].isin(bedroom_counts)][columns_to_show]
print(houses_to_examine)

From the above output, houses with 10 and 11 bedrooms appear to be appropriate based on their `prices`, `bathrooms`, and `sqft_living`. However, the 33 bedroom house has 2 bathrooms and a sale price of $640000 and it must have been a 3 bedroom house recorded as 33.
Therefore, we will replace 33 with 3 bedrooms. 

In [ ]:
# accesing the row index for that row using .loc, 
#we then specify the column and replace as described above.
df1.loc[15856, 'bedrooms'] = 3

To remove other outliers,  we will apply the statistical method, that calculate the z-score of the feature columns. 
The functiion takes in the dataframe , calculates the z-score per column then removes any value that falls within $z-score > |3|$





In [ ]:
# Apply function to df
kc_house_data_clean = remove_outliers(df1)

# saving the clean data to a new dataframe
kc_house_data_clean.to_csv('data/kc_house_data_clean.csv', index=False)

# Reading the now cleaned data again
df = pd.read_csv('data/kc_house_data_clean.csv')
df

### 3.0 Data Exploration
   > for the features

#### 3.1 Investigating Price

We proceed to investigate `price` and its relationship with house features, however since price appears to have outliers, i.e. houses with very high prices and others with very low prices. We will group price into three categories; Upper price, Median price and Low price where, upper price = 75% percentile, median price = 50% percentile and the low price = 25% percentile. 

In [ ]:
# We want to Group price into three cartegories 
# Define the percentile values for each category
price = df['price']
high_percentile = np.percentile(price, 75)
low_percentile = np.percentile(price, 25)

# Group the prices into categories based on the percentiles
high_prices = price[price > high_percentile]
medium_prices = price[(price >= low_percentile) & (price <= high_percentile)]
low_prices = price[price < low_percentile]

# Print the results
#print("High prices:", high_prices)
#print("Medium prices:", medium_prices)
#print("Low prices:", low_prices)

print("Highest Price:", high_prices.max())
print("Median Price:", medium_prices.median())
print("Lowest price:", low_prices.min())

#### 1. How do different priced houses relate with the number  of bedrooms?

In [ ]:
x = df['bedrooms']
y1 = high_prices
y2 = medium_prices
y3 = low_prices

# set plot style
sns.set_style("whitegrid")

# set colors
colors = ["#FFC300", "#DAF7A6", "#FF5733"]

# create figure
fig, ax = plt.subplots(ncols=3, figsize=(14,6), sharey=True)

# bar plot
sns.barplot(x=x, y=y1, color=colors[2], ax=ax[0])
ax[0].set_title("High prices", fontsize=14)
ax[0].set_xlabel("Bedrooms", fontsize=12)
ax[0].set_ylabel("Price ($)", fontsize=12)

# bar plot
sns.barplot(x=x, y=y2, color=colors[1], ax=ax[1])
ax[1].set_title("Median prices", fontsize=14)
ax[1].set_xlabel("Bedrooms", fontsize=12)
ax[1].set_ylabel("Price ($)", fontsize=12)

# bar plot
sns.barplot(x=x, y=y3, color=colors[0], ax=ax[2])
ax[2].set_title("Low prices", fontsize=14)
ax[2].set_xlabel("Bedrooms", fontsize=12)
ax[2].set_ylabel("Price ($)", fontsize=12)

# set title for the whole figure
fig.suptitle("Price vs Bedrooms", fontsize=16)

# adjust spacing
fig.tight_layout()

# show plot
plt.show()



Here, we can see that the larger the number of bedrooms, the higher the prices as well. For average and low-priced houses, there isn't much of an increase but more of constant with a slight increase in prices with the number of bedrooms.

#### 2. How do different priced houses relate with the number  of bathrooms?

In [ ]:
# plotting price against bathrooms
#data
x = df['bathrooms']
y1 = high_prices
y2 = medium_prices
y3 = low_prices

# Histogram plots
# High prices vs No. of bathrooms
sns.set_style("whitegrid")
fig, (ax0, ax1, ax2) = plt.subplots(nrows=3, ncols=1, figsize=(10, 20))

sns.histplot(x=x, y=y1, ax=ax0, color ='green')
plt.title('High-end properties Vs No. of Bathrooms', fontsize=18)
plt.xlabel('No of bathrooms', fontsize=14)
plt.ylabel('High Prices', fontsize=14);

# median price vs No. of bathrooms
#fig, ax1 = plt.subplots( figsize=(10,8))
sns.histplot(x=x, y=y2, ax=ax1, color ='pink')
plt.title('Middle Price Properties Vs No. of Bathrooms', fontsize=18)
plt.xlabel('No of bathrooms', fontsize=14)
plt.ylabel('Middle Prices', fontsize=14);

# low price vs No. of bathrooms
#fig, ax2 = plt.subplots( figsize=(10,8), facecolor ='red')
sns.histplot(x=x, y=y3, ax=ax2, color ='blue')
plt.title('Low-price Properties Vs No. of Bathrooms', fontsize=18)
plt.xlabel('No of bathrooms', fontsize=14)
plt.ylabel('Low Prices', fontsize=14);


In [ ]:
# Number of dataentries per bedrooms
df['bathrooms'].value_counts()

From the Histogram we observed that the majority of median and high-end properties had a higher density at two bathrooms with and aditional bathtub, toilet or sink. 
From the high end we also noticed that the density was more towards the median showing that there is little difference that distinguished them from the median. 
For the low price range the density was more on one bedroomed houses indicating a prevalence of studio apartments.

The histograms suggest that the number of bathrooms is positively correlated with the price of properties. However, we need to perform statistical analysis to confirm this relationship and identify the strength of the relationship.

### 3. How do different-priced houses relate with the seasons?

In [ ]:
#create a dictionary mapping month numbers to month names
import calendar

month_dict = dict(enumerate(calendar.month_name))

# apply the dictionary to the month_sold column to get month names
df['month_name'] = df['month_sold'].apply(lambda x: month_dict[x])

# define seasons based on months
seasons = {'Winter': ['December', 'January', 'February'],
           'Spring': ['March', 'April', 'May'],
           'Summer': ['June', 'July', 'August'],
           'Fall': ['September', 'October', 'November']}

# create a new 'season' column based on the 'month_sold' column
df['season'] = df['month_name'].apply(lambda x: next((season for season, months in seasons.items() if x in months), None))
df['season'].value_counts()

In [ ]:
# Plotting price vs seasons

x = df['season']
y1 = high_prices
y2 = medium_prices
y3 = low_prices


# Scatter plots

# High prices vs No. of bathrooms
fig, (ax0, ax1, ax2) = plt.subplots(nrows=3, ncols=1, figsize=(10, 20))
sns.scatterplot(x=x, y=y1, ax=ax0, color='blue')
plt.title('High-end properties Vs Seasons', fontsize=18)
plt.xlabel('Season', fontsize=14)
plt.ylabel('High Prices', fontsize=14);

# median price vs No. of bathrooms
#fig, ax1 = plt.subplots( figsize=(10,8))
sns.scatterplot(x=x, y=y2, ax=ax1, color='red')
plt.title('Middle Price Properties Vs Seasons', fontsize=18)
plt.xlabel('Season', fontsize=14)
plt.ylabel('Middle Prices', fontsize=14);

# low price vs No. of bathrooms
#fig, ax2 = plt.subplots( figsize=(10,8))
sns.scatterplot(x=x, y=y3, ax=ax2, color='k')
plt.title('Low-price Properties Vs Seasons', fontsize=18)
plt.xlabel('Season', fontsize=14)
plt.ylabel('Low Prices', fontsize=14);

Based on the provided data, the scatter plots show the relationship between the season and the prices of properties in the dataset. From the count of properties sold in each season, it appears that spring and summer are the most popular seasons for home buying, while fall and winter have comparatively fewer sales.

The scatter plots of high-end, medium, and low-price properties against the season show that the prices of properties are generally consistent across all seasons. However, there is a slight increase in the number of high-end properties sold in the spring and a decrease in the number of low-priced properties sold in the winter.

This pattern may be due to various factors, including the perceived desirability of certain seasons for buying and selling homes, the availability of listings during different seasons, or even the preferences of buyers and sellers. For example, the spring season may be associated with better weather conditions and more opportunities for home viewing, which may increase demand for high-end properties. Conversely, the winter season may be associated with lower sales of low-priced properties due to holiday expenses and colder weather. However, further analysis would be necessary to determine the exact reasons for these trends.

### 3.2  Seasons

In [ ]:
# Group by season and month_sold, and count the number of sales for each group
season_sales = df.groupby(['season', 'month_sold'])['price'].count()
# Find the season with the most sales

best_season = season_sales.groupby('season').max()

# Sort the seasons by total sales in descending order
best_season = best_season.sort_values(ascending=False)

# Print the season and corresponding months with the amount sold in each season
for season in best_season.index:
    months = ', '.join(seasons[season])
    sales = season_sales.loc[season]
    print(f"{season} ({months}): {sales.sum()} sales")
    print(sales)

In [ ]:
# Create a bar chart to show the sales for each season
sns.barplot(x=best_season.index, y=best_season.values)
plt.title('Season-wise sales')
plt.xlabel('Season')
plt.ylabel('Number of Sales')
plt.show()

Looking at the sales data, we can see that the spring months of March, April, and May are the best for selling houses with a total of 6518 sales. This is likely due to the fact that Spring is a time of renewal and growth, and people may be more willing to make big changes such as buying a new home during this time. May is the most popular month for selling homes within Spring, with 2414 sales.

Summer, which comprises the months of June, July, and August, had a slightly lower total sales of 6328 compared to spring. July had the highest sales with 2211 followed by June with 2178 and 
August with 1939.

The fall season of September, October, and November had a total of 5056 sales, with October having the highest sales of 1876 followed by September with 1771 and November with 1409.

Lastly, the winter season of December, January, and February had the lowest total sales of 3695. December had the highest sales with 1470 followed by February with 1247 and January with 978. This may be due to the fact that Winter is a time when people tend to stay indoors and may be less inclined to go through the hassle of buying or selling a home.

Overall, it is important to note that this conclusion is based solely on the provided data and may not be reflective of larger trends in the housing market. Additionally, there may be some inaccuracies in the data such as missing or incomplete records, which could affect the accuracy of these findings.


#### 4. How does Location affect the Sale Price of a house?

Location is key when it comes to real estate. Our second objective seeks to understand the geographical distribution of the homes in our dataset and determine where the highest house sales were recorded.

In [ ]:
# Create a copy to manipulate
dfq2 = df.copy()

# Plot scatter plot
plt.figure(figsize = (11,9))
ax = sns.scatterplot(x = dfq2['long'], y = dfq2['lat'], hue = dfq2['price'],
                palette = 'nipy_spectral',s=10, legend = None, data = dfq2)

norm = plt.Normalize(dfq2['price'].min(), dfq2['price'].max())
smap = plt.cm.ScalarMappable(cmap='nipy_spectral', norm=norm)
smap.set_array([])

ax.figure.colorbar(smap, label = 'price in USD')

plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('King County House Sales')
plt.show()

From this visualisation we can already draw initial insights based on the houses' geographical locations. The highest house prices are concentrated in the area with latitude around 47.6 and longitude around -122.25. There is a disparity with southern locations achieving lower house prices.

However there is no context to help us! We would need an underlying map to understand where the cities, highways, bodies of water etc. are located. It is difficult to draw meaningful conclusions with this visualisation alone.

Whilst we could define quadrants using the latitude and longitude, let us seek to plot the houses on a map instead.

Map of house sales To visualise house sales on a map, we will use Bokeh and the built-in map tile from Carto DB. Note that Bokeh requires Mercator coordinates so we will need to convert our latitude and longitude values.


In [ ]:
# Define coord as tuple (lat,long)
dfq2['coord'] = list(zip(dfq2['lat'], dfq2['long']))

# We will create new columns mercator_x and mercator_y to use in our plot.

# Obtain list of mercator coordinates using helper function
mercators = [x_coord(x, y) for x, y in dfq2['coord'] ]

# Create column mercator
dfq2['mercator'] = mercators

# Define columns mercator_x and mercator_y
dfq2[['mercator_x', 'mercator_y']] = dfq2['mercator'].apply(pd.Series)

# Preview relevant columns
dfq2[['zipcode', 'mercator_x', 'mercator_y', 'price']].head()

# Get top 100 prices
dftop = dfq2.sort_values('price', ascending = False)[:100]



In [ ]:

#Plot with follium

# Create a base map by providing coordinates
m = folium.Map(location = [47.5, -122.2])

# Add data for heatmap
heat_data = dftop['coord']

# Add heatmap onto map
HeatMap(heat_data).add_to(m)

# Show map
m

We can also pinpoint these directly using the coordinates. To increase readability, we have chosen to only display the top 30 houses.

In [ ]:
# Plot top 30 houses on map

# Create a base map by providing coordinates
m = folium.Map(location = [47.5, -122.2])

# Add markers for houses
for coordinates in dftop['coord'][:30]:
    folium.Marker([coordinates[0], coordinates[1]], popup = coordinates).add_to(m)

# Show map
m

The above codes may slow down an old machine therefore we created a seperate plot using a lighter package called plotly.


We can the overlay the above data on a topographical map to view the locations and observe additional features
This proved useful as we are able to clearly see the areas where the high end properties were

In [ ]:
import plotly.graph_objs as go

# Define the map layout
layout = go.Layout(
    title='Housing Sales by Location',
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        style='stamen-terrain',
        bearing=0,
        center=dict(
            lat=df['lat'].median(),
            lon=df['long'].median()
        ),
        pitch=0,
        zoom=10
    ),
)

# Define the map data as a scatter plot of the coordinates
data = go.Scattermapbox(
    lat=df['lat'],
    lon=df['long'],
    mode='markers',
    marker=dict(
        size=5,
        color=df['price'],
        
        opacity=0.8
    ),
    text=['Price: ${}'.format(i) for i in df['price']],
    hovertext = df.apply(lambda x: f"Price: ${x['price']}, Waterfront: {x['waterfront']}", axis=1),
)

# Create the map figure and show it
fig = go.Figure(data=[data], layout=layout)
fig.show()

Based on the plotted graph, it is evident that a significant number of high-end residences are situated in the Bellevue area as well as along the waterfront. The region comprises of a considerable size of uninhabitable mountainous terrain, which concentrates most of the housing in the towns. As a result, these towns appear to have a higher density of housing compared to the surrounding areas. 

Additionally, it is important to note that the plot may not be entirely accurate and there may be some data points that are not a representative of the actual situation. For example, some properties may be incorrectly labeled as having waterfront access when they do not, which could affect their price and skew the distribution shown in the plot. 

However, it is important to note that this statement is limited to the data presented in the plot and may not necessarily represent the overall distribution of residences in the area. Further analysis and data may be required to make more conclusive observations.

### 3.3 Waterfront

### 5. Does the Presence of a waterfront feature affect its price?

From the maps, it is apparent that being on the waterfront is highly sought after and many of the most expensive houses in our dataset have this feature. We have been provided with a waterfront feature, which characterises houses which have a view of a waterfront. Let us investigate how this feature relates to price by using a boxplot.

In [ ]:
# Plot boxplot of waterfront feature
sns.boxplot(x = dfq2['waterfront'], y = dfq2['price'])
plt.title("Boxplot of waterfront feature vs. price")
plt.ylabel("price in USD")
plt.xlabel(None)
plt.xticks(np.arange(2), ('Not view of waterfront', 'Waterfront view'))
plt.show()

In [ ]:
waterfrontmean = dfq2[dfq2['waterfront'] == 1]['price'].mean()
nonwaterfrontmean = dfq2[dfq2['waterfront'] == 0]['price'].mean()
print(f"The mean house price for a house with waterfront view is USD {round(waterfrontmean,2)}")
print(f"The mean house price for a house without waterfront view is USD {round(nonwaterfrontmean,2)}")

In [ ]:
len(dfq2[dfq2['waterfront'] == 1])/len(dfq2)*100

From this analysis we can see that most houses in King County have no waterfront.
However, the houses with waterfront have some of the highest prices. Confirming our assumtion from the plot before.

### 3.3 Zipcode

In [ ]:
# Explore zipcode
df['zipcode'].value_counts()

There are 70 unique zipcodes represented in our dataset.

Let us create a scatter plot to explore the median house price per zipcode and see if we can establish zipcode 'tiers'. We will make use of a geojson file showing the zip code boundaries and extract price and zipcode data from our dataset.

In [ ]:
# Extract price and zipcode from our dataset
dfrel = df[['zipcode', 'price']]

# Groupby zipcode and take median price
dfrel = dfrel.groupby(dfrel['zipcode']).median()

# Reset index after groupy
dfrel = dfrel.reset_index()

# Change type to str
dfrel['zipcode'] = dfrel['zipcode'].astype(str)

# Rename column to match type
dfrel = dfrel.rename(columns = {'zipcode' : 'ZIPCODE'})

# Check
dfrel

In [ ]:
# Plotting Zipcode


# Count the number of occurrences of each zipcode
zipcode_counts = dfrel['ZIPCODE'].value_counts()

# Get the 10 most common zipcodes
top_zipcodes = zipcode_counts.head(10).index

# Filter the data to include only the top zipcodes
filtered_df = dfrel[dfrel['ZIPCODE'].isin(top_zipcodes)]

# Set the Seaborn style
sns.set_style("darkgrid")

# Create the scatter plot
sns.scatterplot(x="ZIPCODE", y="price", data=filtered_df, s=100, color="purple")

# Add labels and title
plt.xlabel("ZIPCODE", fontsize=14)
plt.ylabel("Price", fontsize=14)
plt.title("House Prices by Zipcode", fontsize=16)

# Rotate x-axis labels
plt.xticks(rotation=90)

# Show the plot
plt.show()



## 4.0 Data Analysis

### 4.1 Modeling

The objective of this section is to develop a model that accurately predicts sales price of properties in King County, when given various exogenous variables.
The modeling procedure encompasses the selection of suitable algorithms, feature engineering, fine-tuning of hyperparameters, and evaluation of the model's performance.

### Test for Linearity

Based on the data analysis, it appears that most of the variables have a linear relationship to the price, and their relationship appears to be homoscedastic. However, there are some variables, including sqft_lot, floors, condition, latitude, longitude and sqft_lot15, that do not seem to have a linear relationship with price. 

In [ ]:
X = df.drop(columns=['price', 'season', 'month_name', 'month_sold', 'year_sold', 'grade', 'condition', 'view', 'waterfront', 'zipcode'], axis=1)
fig, axs = plt.subplots(3, 4, figsize=(16, 16), sharey=True)
for i in range(3):
    for j in range(4):
        if i*4 + j < len(X.columns):
            sns.regplot(x=X.iloc[:,i*4+j], y='price', data=df, ax=axs[i][j])
            axs[i][j].set_title(X.columns[i*4+j])
plt.tight_layout()
plt.show()

* Multi-collinearity

In [ ]:
numeric_cols = df.select_dtypes(include='number').columns.drop('price')
corr_matrix = df[numeric_cols].corr()

sns.heatmap(corr_matrix, center=0)
plt.savefig('Images/multi-collinearity.png')


## Baseline Model Generation 

In [ ]:
# Looking at correlations between other variables and price
df.corr()["price"]

In [ ]:
# Since Sqft_living is the feature with the strongest correlation in our dataset let's build our baseline model with that
df.plot.scatter(x="sqft_living", y="price");

There appears to be a fairly clear linear relationship between the Square footage of living space feature and price of a home in King County. We can use this to make a baseline model.

In [ ]:
# Define parameters
y = df["price"]
X_baseline = df[["sqft_living"]]

In [ ]:
baseline_results = fit_simple_linear_reg(df, 'price', 'sqft_living')
print(baseline_results)

In [ ]:
# display model coefficients
baseline_results.params

## Baseline Model results

Looking at our summary above, our regression line is
 $$ Price = $$33,399 +  239 * sqft\_living  

We can make the following observations from the results:

* The model is statistically significant with a p-value below 0.05.
* The model explains 40.1% of the variance in price, as indicated by the adjusted R-squared value.
* Both the intercept (const) and the slope (sqft_living) coefficients are statistically significant with t-statistic p-  values well below 0.05.
* For a house with a living space of 1000 square feet, we would expect the price to be about $272399.

* Generally speaking, as the living space of the house increases, so does the price of the house. 




### 5.2 Baseline Model visualization

In [ ]:
fig, ax = plt.subplots()
df.plot.scatter(x="sqft_living", y="price", label="Data points", ax=ax)
sm.graphics.abline_plot(model_results=baseline_results, label="Regression line", ax=ax, color="red")
ax.legend();

In [ ]:
# create partial regression plots for the predictor

fig = plt.figure(figsize=(15,8))
sm.graphics.plot_regress_exog(baseline_results, "sqft_living", fig=fig)
plt.show()

# Multiple Linear Regression Model

In [ ]:
## model 1
predictor_cols = ['bedrooms', 'bathrooms', 'sqft_living', 'floors', 'waterfront', 'condition', 'month_sold','age','sqft_lot','yr_renovated'] 
multiple_model = fit_multiple_linear_reg(df, 'price', predictor_cols)
print(multiple_model)

The  R-squared value (0.5312489269063179), indicates that the model explains 53% of the variations in response variable.

The test training R-squared value indicates that the model is likely overfitting to the training data and not generalizing well to new data.

These results suggest that the multiple linear regression model may have overfit the training data, leading to poor performance on the test set. Further analysis and possibly model tuning may be needed to improve the model's performance.

In [ ]:
predictor_cols1 = ["sqft_living", "bedrooms","bathrooms","sqft_lot","floors","yr_renovated", "waterfront", "zipcode"]
multiple_model1 = fit_multiple_linear_reg(df, 'price', predictor_cols1)
print(multiple_model1)

These results suggest that the multiple linear regression model may have overfit the training data, leading to poor performance on the test set. 


In [ ]:
# polynomial regression
predictor_cols = ['sqft_living']
degree = 2
poly_test = fit_polynomial_reg(df, 'price', predictor_cols, degree)

These results suggest that the polynomial regression model may have overfit the training data, leading to poor performance on the test set. 

The reason for performing polynomial regression is to capture more complex, nonlinear relationships between the predictor and target variables.

In simple linear regression, you model the relationship between the target and a single predictor variable using a straight line. However, in many real-world scenarios, the relationship between the target and the predictor may not be linear, but rather curvilinear or have some other shape.
Polynomial regression addresses this issue by adding polynomial features to the model, which allow it to capture more complex relationships between the predictors and the target variable. By doing so, it can improve the model's fit to the data and potentially lead to better predictive performance.


## Model 2

In [ ]:
# Initial feature colums
predictor_cols = ['bedrooms', 'bathrooms', 'sqft_living', 'floors', 'waterfront', 'zipcode']
predictor_cols_results = fit_simple_linear_reg(df, 'price', predictor_cols)
print(predictor_cols_results)

Looking at our summary above, our regression line is
 $$ Price = -5.078e+07-4.645e+04 * bedrooms+2.991e+04 bathrooms + 254.5901 sqft_living + 8277.1965 floors + 6.573e+05 waterfront+518.6152 zipcode

In this case, the R-squared value is 0.456, which means that the independent variables explain about 45.6% of the variance in house prices.

The second section of the output provides the coefficients and standard errors of the independent variables. The coefficient represents the change in the dependent variable that is associated with a one-unit change in the corresponding independent variable, holding all other variables constant.

The constant term is -5.078e+07, which is the estimated value of the dependent variable when all independent variables are zero.

For the remaining variables:

A one-unit increase in 'bedrooms' is associated with a decrease of $46,450 in house prices, holding all other variables constant.
A one-unit increase in 'bathrooms' is associated with an increase of $29,910 in house prices, holding all other variables constant.
A one-unit increase in 'sqft_living' is associated with an increase of $254.59 in house prices, holding all other variables constant.
A one-unit increase in 'floors' is associated with an increase of $8,277.20 in house prices, holding all other variables constant.
A house located in a waterfront area has a mean house price that is $657,300 higher than a house not located in a waterfront area, holding all other variables constant.

In this output, all independent variables have p-values less than 0.05, indicating that they are statistically significant predictors of house prices.


# Model 3

In [ ]:
# Moving to our second list of features to see if it will improve. 
predictor_cols1 = ["sqft_living", "bedrooms","bathrooms","sqft_lot","floors","sqft_above","yr_renovated", "waterfront", "zipcode"]
predictor_cols1_results = fit_simple_linear_reg(df, 'price', predictor_cols1)
print(predictor_cols1_results)

The R-squared value of 0.464 indicates that the model explains 46.4% of the variance in the dependent variable. The adjusted R-squared value is the same, which suggests that adding or removing explanatory variables does not significantly improve the fit of the model.

The P-values (P>|t|) indicate the statistical significance of each variable. A low P-value (typically below 0.05) suggests that the variable is statistically significant and has a significant effect on the dependent variable.

The intercept or constant term is -4.143e+07, which represents the estimated value of the dependent variable when all independent variables are equal to zero.

The "const" coefficient represents the intercept or constant term, which is the estimated value of the dependent variable when all independent variables are equal to zero. In this case, the intercept is -4.143e+07, which means that if all other independent variables are zero, the estimated value of "price" is negative 41,430,000 dollars. However, since it's not possible for all independent variables to be zero in practice, the intercept should be interpreted with caution.

The "sqft_living" coefficient is 272.8535, which means that a one-unit increase in "sqft_living" is associated with an increase in "price" of 272.8535 dollars, holding all other variables constant.

The "bedrooms" coefficient is -4.704e+04, which means that a one-unit increase in "bedrooms" is associated with a decrease in "price" of 47,040 dollars, holding all other variables constant.

The "bathrooms" coefficient is 2.414e+04, which means that a one-unit increase in "bathrooms" is associated with an increase in "price" of 24,140 dollars, holding all other variables constant.

The "sqft_lot" coefficient is -0.9808, which means that a one-unit increase in "sqft_lot" is associated with a decrease in "price" of 0.9808 dollars, holding all other variables constant.

The "floors" coefficient is 1.286e+04, which means that a one-unit increase in "floors" is associated with an increase in "price" of 12,860 dollars, holding all other variables constant.

The "sqft_above" coefficient is -19.5153, which means that a one-unit increase in "sqft_above" is associated with a decrease in "price" of 19.5153 dollars, holding all other variables constant.


The "waterfront" coefficient is 6.438e+05, which means that a property located on the waterfront is associated with an increase in "price" of 643,800 dollars, holding all other variables constant.



Overall, the model appears to have a good fit based on the high F-statistic and low Prob (F-statistic) value, indicating that the explanatory variables as a whole are statistically significant in explaining the dependent variable. However, it is important to further evaluate the model's assumptions and check for potential issues such as multicollinearity, heteroscedasticity, or outliers.

## Model 4

In [ ]:
# Now our model will match same number of parameters as the first multimodel 
predictor_cols2 = ["sqft_living","bathrooms","floors","yr_renovated", "waterfront", "zipcode"]
predictor_cols2_results = fit_simple_linear_reg(df, 'price', predictor_cols2)
print(predictor_cols2_results)

The constant term is -5.076e+07, which represents the estimated value of "price" when all the independent variables are zero.

For "sqft_living", the coefficient is 226.2587, which indicates that, on average, a one-unit increase in square footage of living space is associated with an increase of $226.26 in the price of the house.

For "bathrooms", the coefficient is 1.918e+04, which indicates that, on average, a one-unit increase in the number of bathrooms is associated with an increase of $19,180 in the price of the house.

For "floors", the coefficient is 1.462e+04, which indicates that, on average, a one-unit increase in the number of floors is associated with an increase of $14,620 in the price of the house.

For "yr_renovated", the coefficient is 63.1908, which indicates that, on average, a one-unit increase in the year of renovation is associated with an increase of $63.19 in the price of the house.

For "waterfront", the coefficient is 6.643e+05, which indicates that, on average, a house with a waterfront location is associated with an increase of $664,300 in the price of the house compared to a house without a waterfront location.

In summary, the model suggests that the square footage of living space, number of bathrooms, number of floors, year of renovation, and waterfront location are significant predictors of the house price.

Overall, the model explains 45.1% of variations in response variable. It appears a good fit since its statistically significant even though it explains low percentage of variations.

### Plotting Residuals for multiple models

In [ ]:
import matplotlib.pyplot as plt

In [ ]:

def plot_residuals_all(results, predictor_cols):
    """
    Plots the residuals for all predictor variables against their observed values.

    Parameters:
    results (OLSRegressionResults): the results of a fitted linear regression model
    predictor_cols (list of str): the names of the predictor columns

    Returns:
    None
    """
    fig, axs = plt.subplots(len(predictor_cols), 1, figsize=(8, 3*len(predictor_cols)), sharex=False)

    for i, col in enumerate(predictor_cols):
        axs[i].scatter(results.model.exog[:, i+1], results.resid, alpha=0.5)
        axs[i].set_xlabel(col)
        axs[i].set_ylabel('Residual')

    plt.tight_layout()
    plt.show()


#### Model Two

In [ ]:

predictor_cols = ['bedrooms', 'bathrooms', 'sqft_living', 'floors', 'waterfront', 'zipcode']
plot_residuals_all(predictor_cols_results, predictor_cols)


#### Model Three

In [ ]:

predictor_cols1 = ["sqft_living", "bedrooms","bathrooms","sqft_lot","floors","sqft_above","yr_renovated", "waterfront", "zipcode"]
plot_residuals_all(predictor_cols1_results, predictor_cols1)


### Model Four

In [ ]:
predictor_cols2 = ["sqft_living","bathrooms","floors","yr_renovated", "waterfront", "zipcode"]

plot_residuals_all(predictor_cols2_results, predictor_cols2)

### Plotting Fitted Values for Multiple Models

In [ ]:

def plot_fitted_values_all(results, predictor_cols):
    """
    Plots the fitted values for all predictor variables against their observed values.

    Parameters:
    results (OLSRegressionResults): the results of a fitted linear regression model
    predictor_cols (list of str): the names of the predictor columns

    Returns:
    None
    """
    fig, axs = plt.subplots(len(predictor_cols), 1, figsize=(8, 3*len(predictor_cols)), sharex=False)

    for i, col in enumerate(predictor_cols):
        axs[i].scatter(results.model.exog[:, i+1], results.fittedvalues, alpha=0.5)
        axs[i].set_xlabel(col)
        axs[i].set_ylabel('Fitted Value')

    plt.tight_layout()
    plt.show()

#### Model Two

In [ ]:
predictor_cols = ['bedrooms', 'bathrooms', 'sqft_living', 'floors', 'waterfront', 'zipcode']
predictor_cols_results
plot_fitted_values_all(predictor_cols_results, predictor_cols)

#### Model Three

In [ ]:
predictor_cols1 = ["sqft_living", "bedrooms","bathrooms","sqft_lot","floors","sqft_above","yr_renovated", "waterfront", "zipcode"]
predictor_cols1_results
plot_fitted_values_all(predictor_cols1_results, predictor_cols1)

#### Model Four

In [ ]:
# Extracting the fit
plot_fit = predictor_cols2_results.fittedvalues

# Plotting the fitted values
fig, axs = plt.subplots(ncols=len(predictor_cols2), figsize=(20, 4))

for i, col in enumerate(predictor_cols2):
    axs[i].scatter(df[col], plot_fit, alpha=0.5)
    axs[i].set_xlabel(col)
    axs[i].set_ylabel('Fitted Value')

plt.tight_layout()
plt.show()

## Model Valuation

Overall, Model 3 appears to be a better model for predicting house prices in King County

In [ ]:
## rmse ## metrics?
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Fit a linear regression model on the training data
lr = LinearRegression()
lr.fit(X_train, y_train)
# Predict the target variable on the testing data
y_pred = lr.predict(X_test)
# Calculate the RMSE
rmse = calculate_rmse(y_test, y_pred)
print('RMSE:', rmse)

The RMSE value of 186162.44586536638 indicates that the root mean square difference between the predicted and actual values is approximately 186162.45 units. The units will depend on the scale of the target variable, but it is typically the same as the units of the target variable.

A lower RMSE value indicates better performance, as it means the model has a smaller difference between the predicted and actual values. Conversely, a higher RMSE value indicates worse performance, as it means the model has a larger difference between the predicted and actual values.

It's important to note that the interpretation of RMSE values depends on the context of the problem and the scale of the target variable. It's often useful to compare the RMSE of different models or the same model with different sets of parameters to determine which one performs better.

### One Hot Encoding 

In [ ]:
df.head()

## model applying One Hot Encoding


In [ ]:
# Load the data
df = df

# Separate the target variable
y = df['price']
X = df.drop('price', axis=1)

# One-hot encode categorical variables
categorical_cols = ['waterfront', 'view', 'condition', 'grade']
encoder = OneHotEncoder(drop='first')
X_encoded = encoder.fit_transform(X[categorical_cols])
X_encoded = pd.DataFrame(X_encoded.toarray(), columns=encoder.get_feature_names(categorical_cols))

# Scale numeric variables
numeric_cols = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'sqft_above', 'sqft_basement', 'age', 'renovated']
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X[numeric_cols])

# Combine one-hot encoded and scaled numeric variables
X_processed = pd.concat([X_encoded, pd.DataFrame(X_scaled, columns=numeric_cols)], axis=1)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)

# Train the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Evaluate the model
print('Training score:', model.score(X_train, y_train))
print('Testing score:', model.score(X_test, y_test))


We can observe that the model having added cartegorical variables by method of OHE(one hot encoding) which applies to the categorical variables and standard scaling to the numerical variables. replacing each cartegory with 1 where is appears and the rest 0. creating separate columns of that variable.

The training score and testing score are measures of how well the linear regression model is able to predict the target variable (price) based on the input features (bedrooms, bathrooms, etc.).

The training score is the R-squared value of the model on the training set. R-squared is a statistical measure that represents the proportion of the variance in the target variable that is explained by the input features. In this case, the training score of 0.529 indicates that the input features explain 52.9% of the variance in the target variable in the training set.

The testing score is the R-squared value of the model on the testing set. This score is a measure of how well the model generalizes to new, unseen data. In this case, the testing score of 0.537 indicates that the input features explain 53.7% of the variance in the target variable in the testing set.

Since the training and testing scores are relatively close to each other, it suggests that the model is not overfitting (performing well on the training data but poorly on the testing data). However, the overall score is relatively low, which indicates that the input features do not explain a large proportion of the variance in the target variable.


### PCA
(Principal Component Analysis) is a widely used statistical technique for dimensionality reduction of high-dimensional data. It aims to transform the original data into a new, lower-dimensional feature space while preserving as much of the original variation or structure in the data as possible.

PCA is used for dimensionality reduction of numerical data by transforming the data into linearly uncorrelated principal components.

In [ ]:
from sklearn.svm import SVC

# Load the data
df = df

# Separate the target variable
# = df['price']
# = df.drop(index=row_index)


# Scale the features
#caler = StandardScaler()
#_scaled = scaler.fit_transform(X)

# Apply PCA to reduce the dimensionality
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# Use the transformed features X_pca to fit your model.
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2)

# Train the linear regression model
svm = SVC(kernel='linear', C=1)
svm.fit(X_train, y_train)


# Evaluate the model
svm.score(X_test, y_test)

### Final Model with desired $R^2$ value
#### Improved OHE 

In [ ]:
def preprocess_data(df):
    # Select all numeric columns except price
    numeric_cols = [col for col in df.columns if col != 'price' and np.issubdtype(df[col].dtype, np.number)]
    
    # Fill missing values with column mean
    df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())
    
    # One-hot encode categorical columns
    cat_cols = [col for col in df.columns if df[col].dtype == 'object']
    df_encoded = pd.get_dummies(df, columns=cat_cols, drop_first=True)
    
    # Standardize the data
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(df_encoded[numeric_cols])
    
    # Combine scaled data and categorical data
    X = np.concatenate([X_scaled, df_encoded.drop(numeric_cols, axis=1).values], axis=1)
    
    # Get target variable
    y = df['price'].values
    
    return X, y



In [ ]:
X, y = preprocess_data(df)
print(X)
print(y)

The first array(X) is a numeric array of standardized data where each row represents a house in a dataset. The first column represents the standardized values of the first numeric variable, the second column represents the standardized values of the second numeric variable, and so on. The last column, which is not standardized, represents the price of each house.

The second array(y) represents the price of each house in the dataset.

In [ ]:
# fit OLS model
X, y = preprocess_data(df)
model = sm.OLS(y, sm.add_constant(X)).fit()
print(model.summary())
# plot residuals vs fitted values
fig, ax = plt.subplots()
ax.scatter(model.fittedvalues, model.resid)
ax.set_xlabel('Fitted Values')
ax.set_ylabel('Residuals')
ax.set_title('Residuals vs Fitted Values')
plt.show()

The linear regression model above, can predict the prices of houses based on certain features, such as the number of bedrooms, the square footage, the year the house was built, and others.

The first part of the code prepares the data for the model by standardizing the numeric features, meaning that it scales them so that they are all on the same scale and centered around 0. This helps the model to be more accurate.

The second part of the code fits the linear regression model to the data and produces a summary of the results. This summary shows the R-squared value, which is a measure of how well the model fits the data (the closer to 1, the better), as well as the coefficients for each feature, which indicate how strongly each feature affects the price.

The third part of the code creates a scatter plot that shows the relationship between the predicted prices and the actual prices of the houses in the dataset. This helps to visualize how well the model is predicting the prices, and to identify any outliers or areas where the model is particularly inaccurate.

This regression analysis is using the Ordinary Least Squares (OLS) method, which is a statistical technique for predicting the relationship between a dependent variable (y) and one or more independent variables (x1, x2, x3, ... xn).

In this particular case, there are 30 independent variables (x1 to x30), and the results show that the model has a very good fit, with an R-squared value of 1.000. The R-squared value is a measure of how well the model explains the variation in the dependent variable. In this case, the value of 1.000 means that the model explains 100% of the variation in the dependent variable.

The table shows the estimated coefficients for each independent variable, along with their standard errors, t-values, and p-values. The coefficients represent the slope of the line that relates each independent variable to the dependent variable. The t-values and p-values are used to test whether the coefficients are significantly different from zero. A p-value less than 0.05 is typically used to indicate statistical significance.

In summary, this analysis suggests that the independent variables are highly predictive of the dependent variable, and that the model fits the data very well.

# Recommendations

** pointers
>attribute excelent model performance to good data understanding
>as well as keen data clean up
>also consider knowledge derived from initial models. 
>recomdend accept our findings as valid support of our busines questions but specify we are not 100% confident it is an actual representation of larger population/country
>suggest areas of investment considering what we observed from our visualizations 
>write conslusion 